<h1>CS152 Assignment 2: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [1]:
# Import any packages you need here
# Also define any variables as needed

# YOUR CODE HERE (OPTIONAL)

#Now, define the class PuzzleNode:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """
    def __init__(self, Matrix):
        
        self.initialMatrix_length = len(Matrix) # matrix length before processing
        self.InitArray = [] # will be storing the two directions of movement(very important inside Direction_Of_Movement())
        self.dictElement = {}
        self.initNode = None
        self.nodeCount = 0
        self.Matrix = self.processMatrix(Matrix) # will process matrix and give us the processed and initial matrix together in a new array
        self.listOfDirections = []

    def processMatrix(self, Matrix):
        
        save_Init_Matrix = Matrix # saving the initial matrix before processing for later reference
        OutputList = []
        
        if type(Matrix) == list and len(Matrix) < 3: # checking if the matrix is 3 * 3
            
            return
        
        row = 0
        
        for innerArray in Matrix: # iterating into the one dimentional array
            
            col = 0
            
            if type(innerArray) != list or len(innerArray) != len(Matrix): # to make sure we have a square array
                
                return 
            
            for eachVal in innerArray: # individual values of the one dimensional list
                
                if eachVal  not in OutputList and type(eachVal) == int:
                    if eachVal == 0:
                        self.InitArray = [row, col] # the two directions of movement initialized
                        
                    OutputList.append(eachVal)     
                    self.dictElement[eachVal] = [row, col] # creating a key:value map for efficient reference
                    
                else:
                    return 
            
                col+=1
            
            row+=1
            
        self.Matrix = [OutputList, save_Init_Matrix] # the first element is processed Matrix, and second is initial Matrix. 
        
        return self.Matrix # now self.Matrix has both the processed and initial values
    
    def __lt__ (self, valFromHeaps): #without this, tuple comparison in heaps will bring in TypeError. Don't forget to implement this method.
        
        
        pass
                  
                
    def Sort_And_Create_Goal(self, Matrix, initLength): # self-explanatory
        
        res_Array = []
        self.Matrix[0].sort()
        for i in range(0, len(Matrix), initLength):
            res_Array.append(Matrix[i: i + initLength])
            
        return res_Array
    
    
    def Direction_Of_Movement(self):
        list_Of_Moves = [] # will store the directions of movement in form of list of tuples
        
        x = self.InitArray[0]
        y = self.InitArray[1]
        
        if x - 1 >= 0:
            
            list_Of_Moves.append((self.Matrix[1][x-1][y], "South")) # using Matrix[1] bacause I want the initial matrix not the processed
            
        if y - 1 >= 0:
            
            list_Of_Moves.append((self.Matrix[1][x][y-1], "East"))
            
        if x + 1 < self.initialMatrix_length:

            list_Of_Moves.append((self.Matrix[1][x+1][y], "North"))
            
        if  y + 1 < self.initialMatrix_length:

            list_Of_Moves.append((self.Matrix[1][x][y+1], "West"))
            
        self.listOfDirections = list_Of_Moves 
        
        return list_Of_Moves # contains the list of possible moves in form of tuples in the form of (int(num), str(direction))
    
    
    def Swap(self, element, direction):
        
        if (element,direction) in self.listOfDirections and type(element) == int: # to check if the move has not been made before

            self.InitArray, self.dictElement[element] = self.dictElement[element], self.InitArray
            self.dictElement[0] = self.InitArray
            self.Matrix[1][self.dictElement[element][0]][self.dictElement[element][1]] = element
            self.Matrix[1][self.InitArray[0]] [self.InitArray[1]] = 0
    
            self.Direction_Of_Movement()
               
        else:
            
            print("This is not a valid move")
            
            


<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [108]:
# Add any additional code here (e.g. for the memoization extension)

# YOUR CODE HERE (OPTIONAL)


# Misplaced tiles heuristic
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    
    init_Instance = PuzzleNode(state)
    
    number_Of_Misplaced_Tiles = 0

    if init_Instance.Matrix == None: # if we don't have Matrix that can't be processed, cut it short. 
        
        return

    init_Goal = init_Instance.Sort_And_Create_Goal(init_Instance.Matrix[0], init_Instance.initialMatrix_length) # defining goal state
    
    
    if init_Instance.Matrix[1] == init_Instance.Sort_And_Create_Goal(init_Instance.Matrix[0], init_Instance.initialMatrix_length):
        
        # this will run if goal state and initial matric state matches (meaning there are no Misplaced Tiles anymore)
        return number_Of_Misplaced_Tiles
    
    else: # otherwise, we will keep looking until we find one. 
        
        for i in range (len(init_Instance.Matrix[1])):
            
            for j in range(len(init_Instance.Matrix[1])):
                
                init_Goal = init_Instance.Sort_And_Create_Goal(init_Instance.Matrix[0], init_Instance.initialMatrix_length)
                
                if init_Goal[i][j] != init_Instance.Matrix[1][i][j] and init_Instance.Matrix[1][i][j] != 0: # goal state is not reached
                    number_Of_Misplaced_Tiles +=1  # so increment the misplaced number of tiles 
                    
    return number_Of_Misplaced_Tiles
 
                    

# Manhattan distance heuristic
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    
    # here, we are concerned about knowing how many steps it will take for us to reach the goal state
    
    Manhattan_Distance = 0
    
    init_Instance = PuzzleNode(state)
    
    init_Goal = init_Instance.Sort_And_Create_Goal(init_Instance.Matrix[0], init_Instance.initialMatrix_length)
    
    if init_Instance.Matrix[1] == init_Goal:
        return Manhattan_Distance # goal state is reached
    
    else:
        
        for i in range(len(init_Goal)):
            for j in range(len(init_Goal)):
                if init_Goal[i][j] !=0:
                    valForIndexing = init_Goal[i][j] # individual value from the 2D goal state
                     
                    whereAbout = init_Instance.dictElement[valForIndexing] 
                    # fetching elements back from the dictionary using the valForIndexing as key
                    # whereAbout is 1D array with two elements(the two directions of movement we have mapped before)
                    Manhattan_Distance += abs(whereAbout[0] - i) 
                    #subtracting individual values of the goal state and direction of movement from initial matrix
                    #we use absolute value the avoid the effect of adding and subtracting negative values to Manhattan_Distance
                    Manhattan_Distance += abs(whereAbout[1] - j)
                                        
    return Manhattan_Distance
    
    
# Extra heuristic for the extension.  If implemented, modify the function below
def h3(state):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Heuristic distance of the state from its solved configuration
    """
    Solved_Config_Distance = 0
    
    init_Instance = PuzzleNode(state)
    
    
    def SearchGoal(compMatrix):
        outPut_Goal = init_Instance.Sort_And_Create_Goal(init_Instance.Matrix[1], init_Instance.initialMatrix_length)

        for outerVal in range(len(outPut_Goal)):
            for innerVal in range(len(outPut_Goal)):

                if outPut_Goal[outerVal][innerVal] == compMatrix:       
                    return[outerVal, innerVal] 
                else:
                    return []
                
        
    for i in range(init_Instance.initialMatrix_length):
        
        for j in range(init_Instance.initialMatrix_length):
            
            if state[i][j] == 0:
                continue
     
            value_Before_Correction = state[i]
                        
            value_After_Correction = SearchGoal(value_Before_Correction)
            

            if [i, j] == value_After_Correction:
                continue
                
            if len (value_After_Correction) > 0:
                
                if i == value_After_Correction[0]:

                    for eachVal in range(j + 1, len(state)):
                        if value_Before_Correction[j] > state[i][eachVal]:
                            Solved_Config_Distance +=1

                elif j == value_After_Correction[1]:

                    for eachVal in range(i+1, len(state)):

                        if value_Before_Correction[j] > state[eachVal][j]:

                            Solved_Config_Distance +=1

    return Solved_Config_Distance +  h2(state) + h1(state)
            
            
# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2, h3]



<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [118]:

import heapq

import copy

# Main solvePuzzle function.
     
def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """
  
    def helperSolvePuzzle(inputVal):

        if len(inputVal) == 3: # if it is 3 * 3, it needs futher comparison
            
            combinedListVal = [] # will store individual values from the 2D array

            for eachList in inputVal:
                for innerVal in eachList:
                    combinedListVal.append(innerVal)
                    
            modCheckCount = 0

            for i in range(len(combinedListVal) -1):
                
                for j in range(i+1, len(combinedListVal)):
                    if combinedListVal[i] and combinedListVal[j] and combinedListVal[i] > combinedListVal[j]:
                        modCheckCount +=1

            if modCheckCount % 2 == 0: #if modCheckCount is divisible by  2, return True
                
                return True
            
            else:
                return False

        else: # if it is not 3 * 3, it is solvable
            
            return True
        
    listOfFrontier = []
    max_frontier = 0
  
    startingNode = PuzzleNode(state)

    if startingNode.Matrix == None: 
        return (0, 0, max_frontier, None, -1) # error state returning -1
    
    if startingNode.Sort_And_Create_Goal(startingNode.Matrix[0], startingNode.initialMatrix_length) == startingNode.Matrix[1]:
        return (0, 0, max_frontier, None, 0)
    
    if helperSolvePuzzle(state) == False:
        return (0, 0, max_frontier, None, -2) # not solvable state, returning -2
    

    heapq.heappush(listOfFrontier, (heuristic(startingNode.Matrix[1]), startingNode)) # pushing to heap for easier pop() later

    listOfExploredValues = [] # to keep track of explored values
            
    opt_path = [] # stores the optimum path
    
    isSolutionFound = False
    
    
    while len(listOfFrontier) != 0 and isSolutionFound == False:
        
        startingNodeHere = heappop(listOfFrontier)[1]
        
        listOfExploredValues.append(str(startingNodeHere.Matrix[1])) 

        startingNodeHere.Direction_Of_Movement() # to make sure the initArray will have values appended to it. Or we will have empty array.

        
        for i in startingNodeHere.listOfDirections:
             
            copyOfParentNode = copy.deepcopy(startingNodeHere) # copying it now so I can make startingNodeHere a parent later.
            # I am using copy because "Assignment statements in Python do not copy objects" - https://docs.python.org/3/library/copy.html
            
            copyOfParentNode.Swap(i[0], i[1])
            
            if str(copyOfParentNode.Matrix[1]) not in listOfExploredValues:
                
                copyOfParentNode.initNode = startingNodeHere
                
                copyOfParentNode.nodeCount += 1

                if copyOfParentNode.Sort_And_Create_Goal(copyOfParentNode.Matrix[0], copyOfParentNode.initialMatrix_length) == copyOfParentNode.Matrix[1]:
                    
                    extendedNode = copyOfParentNode
                    
                    isSolutionFound = True 
                    
                    break # if solution is found, break out of the loop as there is no need to increment nodeCount
                    
                heapq.heappush(listOfFrontier, (copyOfParentNode.nodeCount + heuristic(copyOfParentNode.Matrix[1]), copyOfParentNode)) # add the node count to heuristic val
            
        if max_frontier < len(listOfFrontier):
            
            max_frontier = len(listOfFrontier)
            
    if extendedNode != None:  
        
        parentNodeNow = extendedNode
        
    else:
        parentNodeNow = copyOfParentNode
        
    while parentNodeNow:
            
        opt_path.append(parentNodeNow.Matrix[1])
            
        parentNodeNow = parentNodeNow.initNode
                      
    return (extendedNode.nodeCount, len(listOfExploredValues), max_frontier, opt_path[::-1], 0)
            
  

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [119]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [120]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [ ]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [ ]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same




# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)



assert(steps_mt == steps_man == 9)

In [ ]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [ ]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

In [ ]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.
dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    print(steps_new, "steps_new")
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
    print(dom) # the dom value is positive 2 out of three times. 
    #assert(dom > 0) 

In [ ]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
    assert(dom > 0)

In [ ]:
## Memoization test - will be carried out after submission